In [4]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [5]:
df = gpd.read_file('data/eviction_count_bg_2021.geojson')

In [14]:
# Select required columns and convert df_evic to eviction count dataframe by CBG
df_nonpay = df[['GEOID','geometry','plaintiff_name', 'case_number', 'amount_filed', 'X', 'Y']]
df_other = df_nonpay.copy()

# Remove rows with 'amount_filed' = 'Not Non-Payment of Rent'
df_nonpay = df_nonpay[df_nonpay['amount_filed'] != 'Not Non-Payment of Rent']

# Select rows with 'amount_filed' = 'Not Non-Payment of Rent'
df_other = df_other[df_other['amount_filed'] == 'Not Non-Payment of Rent']

# Drop 'amount_filed' column
df_nonpay = df_nonpay.drop(columns=['amount_filed'])
df_other = df_other.drop(columns=['amount_filed'])

In [40]:
df_nonpay['plaintiff_name'].nunique()*0.05

711.1

In [31]:
# Extract top 100 eviction filing plaintiff names
df_nonpay_100_filer = pd.DataFrame(df_nonpay.groupby('plaintiff_name')['case_number'].count().sort_values(ascending=False)).reset_index().head(100)

# Extract location information by plaintiff name
df_nonpay_loc = df_nonpay.groupby('plaintiff_name').first().reset_index()[['plaintiff_name', 'X', 'Y']]

# Merge df_100_filer with df_loc
df_nonpay_100_filer = pd.merge(df_nonpay_100_filer, df_nonpay_loc, on='plaintiff_name', how='left')

# Extract top 100 eviction filing plaintiff names (other)
df_other_100_filer = pd.DataFrame(df_other.groupby('plaintiff_name')['case_number'].count().sort_values(ascending=False)).reset_index().head(100)

# Extract location information by plaintiff name (other)
df_other_loc = df_other.groupby('plaintiff_name').first().reset_index()[['plaintiff_name', 'X', 'Y']]

# Merge df_100_filer with df_loc (other)
df_other_100_filer = pd.merge(df_other_100_filer, df_other_loc, on='plaintiff_name', how='left')

In [32]:
df_nonpay_100_filer.shape

(100, 4)

In [33]:
df_other_100_filer.shape

(100, 4)

In [34]:
df_other_100_filer.head()

,plaintiff_name,case_number,X,Y
0,ALDEN SHORT INC,348,-96.611773,32.780678
1,THE MUSE APARTMENTS,246,-96.869554,32.705833
2,MARBLETREE APARTMENTS,150,-96.989431,32.863001
3,THE BRANDT,141,-96.989431,32.863001
4,POLARIS TX16 LLC,134,-96.717660,32.908237


In [35]:
df_nonpay_100_filer.head()

,plaintiff_name,case_number,X,Y
0,ALDEN SHORT INC,1798,-96.892691,32.930098
1,VINEYARDS AT FOREST EDGE,1371,-96.724004,32.909431
2,EYAL DALLAS HOLDINGS LLC,648,-96.649463,32.855178
3,POLARIS TX16 LLC,642,-96.717660,32.908237
4,LAKESIDE AT NORTHPARK,543,-96.768520,32.878125


In [36]:
df_nonpay_100_filer.to_csv('data/df_nonpay_100_filer.csv', index=False)
df_other_100_filer.to_csv('data/df_other_100_filer.csv', index=False)